# MuMoT Demonstration Notebook: Variance-Suppression through Negative Feedback <a class="tocSkip">

## Multiscale Modelling Tool <a class="tocSkip">

*Andreagiovanni Reina, Department of Computer Science, University of Sheffield;
James A. R. Marshall, Department of Computer Science, University of Sheffield*

# Introduction
    
This notebook reproduces results representative of those presented in Reina &amp; Marshall ([2020](#references)), in which it is shown that negative feedback can acts as a mechanism to suppress stochastic fluctuations in a simple collective foraging model.

In [ ]:
import mumot

# Model Definitions

We start with a full model of collective foraging that includes positive feedback, through recruitment, and negative, inhibitory feedback.
Taking collective foraging by honeybees as an exemplar, positive feedback through recruitment occurs via the <i>waggle dance</i> (von Frisch, [1967](#references)), while negative feedback occurs via the <i>stop signal</i> (Nieh, [1993](#references)).

In [ ]:
%%model
$
U -> A : q_A
U -> B : q_B
A -> U : a
B -> U : a
A + U -> A + A : r_a
B + U -> B + B : r_b
A + A -> A + U : z
B + B -> B + U : z
$

In [ ]:
fullmodel = mumot.parseModel(In[2])
fullmodel.show()

We next set a finite system size $N$, since on a short timescale there is a constant pool of foragers. This gives us our foraging model including negative feedback:

In [ ]:
negmodel = fullmodel.substitute('U = N - A - B')
negmodel.showODEs()

Next we 'turn-off' the negative feedback part of the model to give a foraging model with only positive feedback:

In [ ]:
posmodel = negmodel.substitute('z = 0')
posmodel.showODEs()

# Results

In the following, we analyse the models using both infinite-population ODEs, and finite population stochastic simulations.

## Positive-feedback only model

## Negative feedback only model

In [ ]:
int1 = ifdmodel.integrate(initialState = {'B': 0.0, 'U': 1.0, 'A': 0.0},
                          initWidgets={'a':[0.001, 0, 1, 0.001],'r':[50, 0, 50, 1],
                                      'v_{A}':[0.8, 0, 1, 0.05],'v_{B}':[0.4, 0, 1, 0.05],
                                       'd_{A}':[0.5, 0, 1, 0.1],'d_{B}':[0.5, 0, 1, 0.1],
                                      'maxTime':[1,0.1,2,0.1]})

In [ ]:
mc1 = mumot.MuMoTmultiController([ifdmodel.SSA(silent=True), ifdmodel.integrate(silent=True)],
                           shareAxes=True,
                           initialState = {'B': 0.0, 'U': 1.0, 'A': 0.0}, choose_yrange=[0,1],
                          initWidgets={'a':[0.001, 0, 1, 0.001],'r':[50, 0, 50, 1],'h':[0.5, 0, 2, 0.1],
                                      'v_{A}':[0.8, 0, 1, 0.05],'v_{B}':[0.4, 0, 1, 0.05],
                                       'd_{A}':[1, 0, 1, 0.1],'d_{B}':[1.0, 0, 1, 0.1],
                                      'systemSize':[200,10,200,10],
                                       'maxTime':[1,0.1,2,0.1],'runs':[1,1,50,1]})

In [ ]:
int2 = vasimodel.integrate(initialState = {'B': 0.0, 'U': 1.0, 'A': 0.0},
                          initWidgets={'a':[0.001, 0, 1, 0.001],'r':[50, 0, 50, 1],
                                      'v_{A}':[0.8, 0, 1, 0.05],'v_{B}':[0.4, 0, 1, 0.05],
                                       'd_{A}':[0.5, 0, 1, 0.1],'d_{B}':[0.5, 0, 1, 0.1],
                                      'maxTime':[10,0.1,50,1]})

In [ ]:
mc2 = mumot.MuMoTmultiController([vasimodel.SSA(silent=True), vasimodel.integrate(silent=True)],
                           shareAxes=True,
                           initialState = {'B': 0.0, 'U': 1.0, 'A': 0.0}, choose_yrange=[0,1],
                          initWidgets={'a':[0.001, 0, 1, 0.001],'r':[50, 0, 50, 1],'h':[0.5, 0, 2, 0.1],
                                      'v_{A}':[0.8, 0, 1, 0.05],'v_{B}':[0.4, 0, 1, 0.05],
                                       'd_{A}':[1, 0, 1, 0.1],'d_{B}':[1.0, 0, 1, 0.1],
                                      'systemSize':[200,10,200,10],
                                       'maxTime':[10,1,50,1],'runs':[10,1,50,1]})

# References 
<a id='references'></a>
<ul>
<li>Reina, A. &amp; Marshall, J. A. R. (2020) Negative feedback may suppress variation to improve collective foraging performance. <i>In preparation</i>
<li>von Frisch K (1967). <i>The Dance Language and Orientation of Bees</i>. Belknap Press, Cambridge, MA.
<li>Nieh, J. (1993). The stop signal of the honey bee: reconsidering its message. <i>Behav. Ecol. Sociobiol.</i> <b>33</b>, 51–56.
</ul>